In [0]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("Payment Charge Analysis ").getOrCreate()

In [0]:
spark

In [0]:
charge_raw_df=spark.read.format("csv").option("header","true").option("inferSchema","true").load("s3://source-payments-transaction/input/")

In [0]:
charge_raw_df.show()

In [0]:
#remove invalid users
valid_user_df=charge_raw_df.filter("UserId!=-1")

In [0]:
valid_user_df.count()

In [0]:
charge_raw_df.count()

In [0]:
valid_user_df.printSchema()

In [0]:
#changing the timestamp to Date (added a column for date),column name change for `Transaction ID`
valid_user_df.createOrReplaceTempView("VALID_USER")
user=spark.sql("select substring(Timestamp,0,10 ) as Transaction_Date,`Transaction ID` as Transaction_ID,ItemCode, ItemDescription,NumberOfTransaction,Charges,Country  from VALID_USER")
user.show()



In [0]:
user.createOrReplaceTempView("Payment_charge")

In [0]:
user.printSchema()

In [0]:
updated_schema=user.withColumn("Charges",user.Charges.cast("double")).withColumn("NumberOfTransaction",user.NumberOfTransaction.cast("int")).withColumn("ItemCode",user.ItemCode.cast("int"))

In [0]:
updated_schema.printSchema()

In [0]:
updated_schema.show()

In [0]:
updated_schema.write.format("parquet").mode("overwrite").save("s3://source-payments-transaction/output/")


In [0]:
%sql
CREATE EXTERNAL LOCATION my_external_location
URL 's3://source-payments-transaction'
WITH (STORAGE CREDENTIAL databricks_aws_project);

In [0]:
%sql SHOW STORAGE CREDENTIALS;

In [0]:
 %sql USE CATALOG databricks_aws_project;

In [0]:
%sql SHOW CATALOGS;

In [0]:
%sql
create schema if not exists Payments;

In [0]:
%sql
CREATE TABLE databricks_aws_project.Payments.Payment_charge
(
 Transaction_Date string,
Transaction_ID string,
ItemCode integer,
ItemDescription string,
NumberOfTransaction integer,
Charges double,
Country string
)
LOCATION 's3://databricks-aws-project-bucket/unity-catalog/4366105847627198/output/';

In [0]:
%sql
select * from databricks_aws_project.Payments.Payment_charge